Run seqsimla using a variant with a MAF=0.1 and OR=2.0, additionally disease prevalence of 10%. For this I will use as input the reference sequence file `EUR_chr1.bed.gx`, the recombination file `EUR_chr1.rec`, the frequency file `EUR_chr1.freq` and the simulated pedigree file `simped1000.ped`.

In [4]:
[global]
# Name for the outputfiles
parameter: header = 'Sim1'
# Reference sequence file
parameter: popfile = 'EUR_chr1.bed.gz'
# Recombination file
parameter: recfile = 'EUR_chr1.rec'
# Disease prevalence
parameter: prev = 0.1
# Odd ratio
parameter: OR = 2.0
# model --mode-prev for simulating disease status based on prevalence and OR
parameter: model = '--mode-prev'
# Path to the ped file (6-column PED in linkage format)
parameter: famfile = path('simped1000.ped')
# Output directory
parameter: folder = path('results')
# Select location of disease sites
parameter: site = 80
#The number of simulated replicas to generate
parameter: batch = 100

In [7]:
[simulate (SeqSIMLA)]
bash: container = 'dianacornejo/seqsimla' , expand = '${ }'
    mkdir -p ${folder}
    SeqSIMLA -popfile ${popfile} -recfile ${recfile} -famfile ${famfile} -folder ${folder} -header ${header} -batch ${batch} -site ${site} ${model} -prev ${prev} -or ${OR}

In [ ]:
SeqSIMLA -popfile EUR_500K.bed.gz -recfile EUR_500K.rec -famfile simped1000.ped -proband proband.txt -folder results -header Sim1 -batch 1 -site 1 --mode-prev -prev 0.1 -or 2.0

In [ ]:
SeqSIMLA -popfile ASN_500k.bed.gz -recfile ASN_500k.rec -famfile SAP.txt -proband probands.txt -folder test1 -header test -batch 1 -site 1,200,3000 --mode-prev -prev 0.05 -or 1.2 